In [1]:
class EmptyClass:
    """Пустой класс без атрибутов и методов."""

print(EmptyClass)
print(type(EmptyClass))
print(EmptyClass.__doc__)
print(type(int))

<class '__main__.EmptyClass'>
<class 'type'>
Пустой класс без атрибутов и методов.
<class 'type'>


## Объект объявления класса

Инструкции внутри тела класса выполняются сразу после того, как поток программы доберется до объявления класса и интерпретатор прочитает заголовок класса.
В результате создаётся объект объявления класса (не экземпляр класса), который имеет тип type, т.е. этот объект задает новый тип данных.
Этот объект будет использоваться в качестве фабрики для создания экземпляров

Важно понимать, что все инструкции внутри тела класса выполняются в тот момент, когда интерпретатор читает объявление класса, а не когда создаются экземпляры этого класса. В примере ниже объявляется класс, но не создаётся ни один его экземпляр. Инструкция print все равно выполняется.

In [2]:
print(f"До создания класса MyClass")

class MyClass:
    print(f"Создаётся класс MyClass")

print(f"Класс {MyClass.__name__} создан")

До создания класса MyClass
Создаётся класс MyClass
Класс MyClass создан


## Создание экземпляра

In [3]:
e = EmptyClass()
isinstance(e, EmptyClass)

True

Итак, инструкция EmptyClass() создаётся экземпляр класса EmptyClass, который связывается с именем e.

Команда isinstance(e, EmptyClass) демонстрирует, что созданный объект — экземпляр искомого класса. Это возможно сделать в runtime, т.к. несмотря на то,
что тело класса EmptyClass по сути дела пустое, у созданного объекта всегда создаётся специальный атрибут __class__, который ссылается на класс,
к которому относится экземпляр.

In [4]:
e.__class__

__main__.EmptyClass

In [5]:
# Здесь объявляется атрибут x целочисленного типа, который инициализируется при создании объекта. Но в python нельзя объявить переменную
# и не инициализировать её: каждое имя должно ссылаться на какой-то объект. Попробуем построить аналогичное объявление,
# инициализировав атрибут числом 100.

class A:
    x = 100

In [6]:
# Инструкции в теле класса выполняются не в момент создания экземпляра, а в момент чтения объявления класса интерпретатором.
# Т.е. атрибут x должен уже существовать, несмотря на то, что не было создано ни одного экземпляра.
# Чтобы убедиться в этом, проверим наличие атрибута x у объекта объявления класса A, используя точечную нотацию.

print(A.x)

100


In [7]:
# Итак, такое объявление создаёт атрибут самого класса (атрибут объекта объявления класса).
# В терминах C++ это статический атрибут. Иными словами, этот атрибут разделяется всеми экземплярами этого класса.

a1 = A()
a2 = A()

print("До изменения атрибута")
print(A.x, a1.x, a2.x)

A.x = 1

print("После изменения атрибута")
print(A.x, a1.x, a2.x)

До изменения атрибута
100 100 100
После изменения атрибута
1 1 1


При вызове метода через экземпляр класса первым аргументом неявно передаётся ссылка на этот самый экземпляр.
Это позволяет таким методам получать доступ к вызвавшему экземпляру, его атрибутам, методам и т.п.
Однако программистам приходится явно указывать дополнительный параметр в объявлении метода класса на первой позиции.
Имя этого параметра может быть произвольным, но общепринято называть его self.
Этот параметр очень похож на this в C++, но в python приходится явно указывать этот параметр.

In [8]:
class A:
    def f():
        print("Метод f класса A")

A.f()

Метод f класса A


In [9]:
a = A()

try:
    a.f()
except TypeError as msg:
    print(msg)

A.f() takes 0 positional arguments but 1 was given


In [10]:
class A:
    def f(self):
        print("Метод f класса A")

    def g(self, x):
        print(f"Метод g класса A вызван с параметром {x}")

a = A()

a.f()
a.g(42)

Метод f класса A
Метод g класса A вызван с параметром 42


Методы класса — можно считать обычными атрибутами класса.
Они объявляются в теле класса, а значит их объекты создаются во время создания объекта объявления класса, а не во время создания экземпляра.
Однако есть интересная деталь, которая объясняет почему первым параметром передаётся ссылка на вызвавший экземпляр:
все функции являются дескрипторами, т.е. функции — объекты, у которых специализирован метод __get__.
Если атрибут класса является дескриптором, то при получении доступа к нему через экземпляр класса возвращается не сам дескриптор,
а результат вызова его метода __get__.

У всех функций по умолчанию, метод __get__ принимает на вход экземпляр и тип этого экземпляра, а возвращает обертку над этой самой функцией,
которая подставляет экземпляр класса в качестве первого аргумента при вызове функции.
Такая обертка над функцией называется связанным методом (bound method): метод привязан к экземпляру.

Ссылка на исходную функцию хранится в атрибуте __func__ связанного метода.

Продемонстрируем факт того, что A.f и a.f в предыдущем примере — разные сущности.

In [11]:
print(A.f) # <function A.f at 0x...>
print(a.f) # <bound method A.f of <__main__.A object at 0x...>>
print(A.f is a.f) # False
print(A.f is a.f.__func__) # True

<function A.f at 0x000001FB1E19ED40>
<bound method A.f of <__main__.A object at 0x000001FB1E120C40>>
False
True


In [14]:
# Статические методы

class A:
    def my_instance_method(self):
        print("Обычный метод.")

    @staticmethod
    def my_static_method():
        print("Статический метод.")


a = A()

print(a.my_static_method)
print(a.my_instance_method)
A.my_static_method()
a.my_static_method()

<function A.my_static_method at 0x000001FB1E423760>
<bound method A.my_instance_method of <__main__.A object at 0x000001FB1E7DA470>>
Статический метод.
Статический метод.


### Note
Часто возникают сомнения, должен ли метод быть объявлен статическим внутри класса или снаружи в виде обычной функции, если они никак не взаимодействуют с экземпляром. Общего правила на это счет нет. Иногда удобно объявить метод статическим, чтобы поместить его в пространство имен класса и тем самым не только освободить глобальное пространство имен модуля, но и явно указать, что этот метод как-то связан с этим типом данных. Ещё часто статические методы применяют для реализации альтернативных конструкторов: действительно, конструктор класса, по определению не должен принимать на вход экземпляр класса, а должен его возвращать.

In [15]:
# При создании экземпляра класса (вызов объекта объявления класса) сначала вызывается специальный метод __new__ соответствующего класса,
# который именно создаёт объект и возвращает его. Затем у этого уже созданного объекта вызывается специальный метод __init__, который его инициализирует.

class A:
    def __init__(self, x):
        print(f"Инициализируется экземпляр класса A с атрибутом x = {x}")
        self.x = x

a1 = A(3)
a2 = A(42)
print(f"a1.x = {a1.x}\na2.x = {a2.x}")

Инициализируется экземпляр класса A с атрибутом x = 3
Инициализируется экземпляр класса A с атрибутом x = 42
a1.x = 3
a2.x = 42


### Публичные и не публичные поля. Справка по классу
В python нет поистине приватных атрибутов: какие бы меры вы не приняли, будет возможно получить доступ ко всем атрибутам класса.
От части в связи с этим, в python не принято принимать больших усилий для скрытия атрибутов.
Вместо этого существует общепринятое правило, которое отличает публичные имена в классе от не публичных.

Все атрибуты и методы, имена которых начинаются с одинарного нижнего подчеркивания, не считаются публичными.
Разработчик класса не рассчитывает, что к ним будет осуществляться доступ напрямую вне самого класса.

Все атрибуты и методы, имена которых не начинаются с нижнего подчеркивания, считаются публичными.

Так, метод public_method в примере ниже считается публичным, а _private_method нет.

Набор публичных атрибутов методов часто называют интерфейсом класса. Разработчик класса (модуля, библиотеки) берет на себя обязательство поддерживать интерфейс в ближайших обновлениях. Публичные методы документируются. Пользователь может смело получать доступ к публичным атрибутам и вызывать публичные методы.

Остальные атрибуты и методы не входят интерфейс. Разработчик вправе не составлять исчерпывающую документацию для непубличных методов или не документировать их вовсе. Разработчик может в любой момент убрать или изменить роль любого таких атрибутов и методов по своему усмотрению (например, при рефакторинге). Пользователь обращается к ним на свой страх и риск.

В частности, встроенная функция справки help не перечисляет непубличные методы.

In [16]:
class A:
    "docstring of class"
    def public_method(self):
        "docstring of method"

    def _private_method(self):
        pass

help(A)

Help on class A in module __main__:

class A(builtins.object)
 |  docstring of class
 |  
 |  Methods defined here:
 |  
 |  public_method(self)
 |      docstring of method
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



## Треугольник

In [17]:
from math import asin, sqrt, cos, sin, pi


def degree_to_radian(angle):
    return angle * pi / 180

class Triangle:
    "Класс для работы с треугольниками. "
    def __init__(self, AB, BC, CA):
        "Конструктор. В качестве параметров принимает длины 3 сторон."
        if not Triangle.is_valid(AB, BC, CA):
            raise ValueError("Нарушено неравенство треугольника.")

        self._AB = AB
        self._BC = BC
        self._CA = CA

    @staticmethod
    def is_valid(AB, BC, CA):
        "Проверяет выполнение неравенства треугольника."
        if AB + BC < CA:
            return False
        if BC + CA < AB:
            return False
        if CA + AB < BC:
            return False
        return True

    @staticmethod
    def _is_valid_angle(angle):
        if angle <= 0:
            return False
        if angle >= pi:
            return False
        return True

    @staticmethod
    def from_2_edges_and_1_angle(AB, CA,  CAB, degree=False):
        "Конструктор. В качестве параметров принимает 2 стороны и угол между ними."
        if degree:
            CAB = degree_to_radian(CAB)

        if not Triangle._is_valid_angle(CAB):
            raise ValueError("Угол должен быть в интервале от 0 до Pi.")

        BC = sqrt(CA*CA + AB*AB - 2*AB*CA*cos(CAB))
        return Triangle(AB, BC, CA)

    @staticmethod
    def from_1_edge_and_2_angles(AB, ABC, CAB, degree=False):
        "Конструктор. В качестве параметров принимает сторону и прилежащие углы."
        if degree:
            ABC = degree_to_radian(ABC)
            CAB = degree_to_radian(CAB)

        if not Triangle._is_valid_angle(ABC):
            raise ValueError("Угол должен быть в интервале от 0 до Pi.")
        if not Triangle._is_valid_angle(CAB):
            raise ValueError("Угол должен быть в интервале от 0 до Pi.")

        BCA = pi - ABC - CAB
        if not Triangle._is_valid_angle(BCA):
            raise ValueError("Сумма углов должна быть меньше Pi.")

        CA = AB / sin(BCA) * sin(ABC)
        BC = AB / sin(BCA) * sin(CAB)
        return Triangle(AB, BC, CA)

    def perimeter(self):
        "Возвращает периметр треугольника."
        return self._AB + self._BC + self._CA

    def area(self):
        "Возвращает площадь треугольника."
        p = self.perimeter() / 2.
        return sqrt(p*(p - self._AB)*(p - self._BC)*(p - self._CA))

* Треугольник однозначно задаётся длинами его сторон, а значит разумно задать эти длины в качестве атрибутов объекта треугольника.

* Далее встаёт вопрос, считать ли эти атрибуты публичными или нет? Не из любых отрезков можно сложить треугольник.
  Чтобы треугольник с заданным набором длин сторон существовал, необходимо, чтобы выполнялось неравенство треугольника.
  Таким образом, если сделать атрибуты публичными, то пользователь сможет по неосторожности сделать из возможного треугольника невозможный.
  Сделаем эти атрибуты непубличными.

* Треугольник с заданной комбинацией сторон может не существовать, а значит логично будет выделить отдельную процедуру для проверки неравенства треугольника Проверка на существование заданного треугольника по логике должна осуществляться до создания объекта.
  Иначе, в какой-то момент будет существовать невозможный треугольник. По этой причине логично считать, что эта процедура должна в качестве параметров принимать длинны сторон, а не экземпляр класса.
  Выше принято решение сделать такой метод статическим (Triangle.is_valid).
  В качестве альтернативы можно было рассмотреть реализацию этого метода в виде функции снаружи класса.
  При этом, так как такой метод может быть полезен и в других ситуациях, то можно сделать его публичным.

* Стандартный конструктор принимает в качестве параметров длины отрезков. Если отрезки не удовлетворяют неравенству треугольника, то возбуждается исключение ValueError. В обратной ситуации создаётся объект с непубличными атрибуты.

* Два альтернативных конструктора, позволяющие задавать треугольники в виде комбинации сторон и углов, реализованны в качестве статически методов.

* Разумно ограничить все углы в отрезке. Для этого реализован непубличный статический метод _is_valid_angle.

* Публичные методы perimeter и area вычисляют периметр и площадь треугольника соответственно.

In [18]:
t1 = Triangle(3, 4, 5)
t2 = Triangle.from_2_edges_and_1_angle(3, 4, 90, degree=True)

alpha = asin(3 / 5)
beta = asin(4 / 5)
t3 = Triangle.from_1_edge_and_2_angles(5, alpha, beta)

print(t1.area(), t2.area(), t3.area())

6.0 6.0 6.0


In [19]:
help(Triangle)

Help on class Triangle in module __main__:

class Triangle(builtins.object)
 |  Triangle(AB, BC, CA)
 |  
 |  Класс для работы с треугольниками.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, AB, BC, CA)
 |      Конструктор. В качестве параметров принимает длины 3 сторон.
 |  
 |  area(self)
 |      Возвращает площадь треугольника.
 |  
 |  perimeter(self)
 |      Возвращает периметр треугольника.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  from_1_edge_and_2_angles(AB, ABC, CAB, degree=False)
 |      Конструктор. В качестве параметров принимает сторону и прилежащие углы.
 |  
 |  from_2_edges_and_1_angle(AB, CA, CAB, degree=False)
 |      Конструктор. В качестве параметров принимает 2 стороны и угол между ними.
 |  
 |  is_valid(AB, BC, CA)
 |      Проверяет выполнение неравенства треугольника.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined

### Абстрактный базовый класс. Абстрактный метод.
Модуль abc (сокращение от Abstract Base Class) предоставляет инструменты для реализации абстрактных базовых классов, т.е. классов, которые лишь задают интерфейс и не предназначены для создания экземпляров напрямую. Обычно, абстрактный базовый класс наследует от abc.ABC, а абстрактные методы помечаются декоратором abc.abstractmethod. Производные от такого абстрактного базового класса классы смогут создавать экземпляры, только если они переопределят все абстрактные методы. Если не переопределен хоть один из абстрактных методов, то python возбудит ошибку при попытке создать экземпляр. Так как тело абстрактной функции не играет никакой роли, то в нем часто возбуждают исключение NotImplementedError.

В качестве примера реализуем абстрактный базовый класс Shape для геометрической фигуры. Как и в примере с треугольником, будем считать, что каждая фигура должна уметь считать свой периметр и площадь.

In [20]:
from abc import ABC, abstractmethod
from math import pi


class Shape(ABC):
    @abstractmethod
    def perimeter(self):
        raise NotImplementedError

    @abstractmethod
    def area(self):
        raise NotImplementedError


class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def perimeter(self):
        return 2 * pi * self.radius

    def area(self):
        return pi * self.radius * self.radius

c = Circle(1)
print(c.area(), c.perimeter())

3.141592653589793 6.283185307179586


## Полиморфизм в python

**Ad-hoc полиморфизм** пример полиморфизма,
>**Ad-hoc-полиморфизм** (*в русской литературе чаще всего переводится как «специальный полиморфизм» или «специализированный полиморфизм», хотя оба варианта не всегда верны*) поддерживается во многих языках посредством перегрузки функций и методов, а в слабо типизированных — также посредством приведения типов.

который не свойственен для python, т.к. перегружать функции честным образом в нем нельзя: каждое следующее объявление функции с таким же именем затрет предыдущее.

В python нельзя перегружать функции по типу параметров. Лучшее что вы можете сделать — проверить в runtime, какого типа аргумент, и в соответствии с этим проделать необходимые операции.

**singledispatch**
В ряде ситуаций удобно применить **singledispatch** из модуля **functools**, который позволяет как бы перегружать функции по типу первого аргумента.

Сначала объявляется и декорируется наиболее общая функция, которую называют generic. Это приводит к тому, что создаётся объект-обертка с именем этой функции, у которого есть метод-декоратор register. С помощью него регистрируются все остальные реализации этой функции для разного типа первого параметра.

Перегруженные версии функции объявляются с именем, отличным от имени исходной функции. Обычно это "_".

In [21]:
from functools import singledispatch

@singledispatch
def double_it(x):
    """
    Generic function.
    Общая функция.
    Она будет вызвана,
    если тип аргумента не соответствует ни одному зарегистрированному.
    """
    raise TypeError(f"Функция double_it вызвана с неподдерживаемым типом аргумента {type(x).__name__}")

@double_it.register(int)
def _(x):
    return 2 * x


@double_it.register(list)
def _(x):
    return [double_it(value) for value in x]

print(double_it(1))
print(double_it([1, 2, 3]))

2
[2, 4, 6]
